In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

이 코드는 Transformer 모델을 학습할 때, 한 번의 배치(batch)에 포함된 데이터를 다루는 Batch 클래스입니다.  
핵심은 마스크(mask)를 생성해서 모델이 패딩 토큰을 무시하고, 디코더가 미래의 단어를 보지 않도록 제한하는 것입니다.  
아래에 주요 부분을 단계적으로 설명드릴게요.

In [2]:
class Batch:
    """Object for holding a batch of data with mask during training."""

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask

## 주요 속성 및 역할
### 1. self.src_mask
``` python
self.src_mask = (src != pad).unsqueeze(-2)
```

소스 시퀀스에서 패딩(pad)이 아닌 위치를 True로 표시하는 마스크입니다.  
.unsqueeze(-2)는 형태를 (batch_size, 1, seq_len)으로 만들어 멀티헤드 어텐션에서 broadcasting이 가능하게 합니다.

In [ ]:
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)